In [36]:
import tensorflow as tf
import sys
sys.path.append("..")
import tools
import numpy as np
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
dataset_test = tf.data.TFRecordDataset(["/astro/users/kmrakovc/Projects/LSST_Streak_Detection/DATA/test.tfrecord"])
tfrecord_shape = tools.model.get_shape_of_quadratic_image_tfrecord(dataset_test)
dataset_test = dataset_test.map(tools.model.parse_function(img_shape=tfrecord_shape, test=False))
dataset_test = tools.data.tfdataset_merge (dataset_test, (4176, 2048))
truths = tools.data.dataset_to_numpy(dataset_test)

In [15]:
dataset_test = tf.data.TFRecordDataset(["/astro/users/kmrakovc/Projects/LSST_Streak_Detection/DATA/test.tfrecord"])
tfrecord_shape = tools.model.get_shape_of_quadratic_image_tfrecord(dataset_test)
dataset_test = dataset_test.map(tools.model.parse_function(img_shape=tfrecord_shape, test=True))
dataset_test = dataset_test.batch(128)

In [30]:
model = tf.keras.models.load_model("../DATA/Trained_model" , custom_objects={'F1_Score':tools.model.F1_Score})
predictions = model.predict(dataset_test)
predictions = tools.data.npy_merge(predictions>0.5, (4176, 2048))

240/240 [==============================] - 6s 26ms/step


2024-03-27 07:36:07.016838: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 6444269715036506559


In [48]:
TP = (predictions==truths) & (predictions==1)
TN = (predictions==truths) & (predictions!=1)
FP = (predictions!=truths) & (predictions==1)
FN = (predictions!=truths) & (predictions!=1)
F1 = TP.sum()/(TP.sum()+0.5*(FP.sum()+FN.sum()))
print ("Pixel wise F1 score is:", F1)

Pixel wise F1 score is: 0.3851867598694494


In [ ]:
TP_a = np.array([tools.data.get_asteroid_num(img) for img in TP])
print ("Asteroid wise TP is:", TP_a.sum())
TN_a = np.array([tools.data.get_asteroid_num(img) for img in TN])
print ("Asteroid wise TN is:", TN_a.sum())
FP_a = np.array([tools.data.get_asteroid_num(img) for img in FP])
print ("Asteroid wise FP is:", FP_a.sum())
FN_a = np.array([tools.data.get_asteroid_num(img) for img in FN])
print ("Asteroid wise FN is:", FN_a.sum())

Asteroid wise TP is: 621


In [50]:
TP_a

[5,
 16,
 4,
 7,
 9,
 11,
 11,
 12,
 16,
 7,
 5,
 7,
 18,
 11,
 15,
 11,
 12,
 15,
 11,
 8,
 8,
 10,
 16,
 7,
 11,
 9,
 16,
 10,
 6,
 5,
 13,
 10,
 13,
 12,
 21,
 9,
 12,
 14,
 11,
 13,
 11,
 9,
 13,
 13,
 12,
 15,
 9,
 10,
 12,
 6,
 16,
 11,
 2,
 11,
 7,
 4,
 10,
 13]